In [ ]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime
import re
import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
import simpler_sf

simpler_sf.simple_salesforce()

In [ ]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [ ]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

Descarga de empresas en sistema


In [ ]:
llamadaEmpresaAntiguas = "SELECT Id,Empresa__r.Id, Empresa__r.Name, Empresa__r.Holding__c, Empresa__r.Campa_a__c, Empresa__r.Id_Empresa__c, Empresa__r.Empleador__c, Empresa__r.poliza__c, Empresa__r.RUT_Empresa__c, Empresa__r.Corredor__c,Empresa__r.Padre_Empresa__c, Name FROM Programas_por_Empresas__c WHERE Empresa__r.Padre_Empresa__c = false"
vllamadaEmpresa = sf.smart_query(llamadaEmpresaAntiguas)
vllamadaEmpresa.to_excel("F:/Empresas SF/reportes/empresas_Antiguas.xlsx", index=False)
vllamadaEmpresa.head()

vllamadaEmpresa.shape


In [ ]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false  
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

In [ ]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

In [ ]:
# Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c": "Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
    ]
]

Programas_empresas.shape

In [ ]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx", index=False)

In [ ]:
query2 = "select P_liza__c, Nombre_de_Empresa__c, Rut_Empresa__c, Empresa_Padre__r.Name, Empresa__r.name,Holding__c, Plan__c, Pais__c, Activo__c, Id from Datos_Empresa__c "

df2 = sf.smart_query(query2)
df2["LLave"] = df2["Rut_Empresa__c"].str.upper().str.replace("-", "") + df2["P_liza__c"]
df2["dato"] = 1
df2.to_excel("F:/Empresas SF/reportes/empresas_nuevas.xlsx", index=False)
df2.head()

fin empresas y comienzo de rescate de pacientes


In [ ]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True 
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.


In [11]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape

(1545470, 4)

In [12]:
# Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c",
)
empresas_unidas.shape

(1545470, 13)

In [13]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


Carga de Archivo carga


In [14]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "Vida Security"
Pais = "Chile"
# Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo.head(10)

,Llave,Cog Convenio,Nombre Convenio,Poliza,Rut Contratante,Dv Contratante,Nombre Contratante,Rut Filial,Dv Filial,Empresa,...,rut2,Rut Asegurado,Dv Asegurado,rut1,Parentesco,nombres,apellidos,nacimiento,sexo,email
0,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,10025371-2,10025371,2,10025371-2,TITULAR,VIVIANA,CORREA ELIAS,1967-03-04 00:00:00,F,VIVIANA.E.CORREA@GMAIL.COM
1,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,10028831-1,10028831,1,10028831-1,TITULAR,DANIELLI JOEL,GODOY JOFRE,1966-10-29 00:00:00,M,DANIELLI.DD.GODOY@BHP.COM
2,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,10028831-1,10934210,6,10934210-6,CÓNYUGE,RODE ESTER,GONZALEZ CISTERNAS,1968-06-09 00:00:00,F,NaN
3,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,10035740-2,10035740,2,10035740-2,TITULAR,EDUARDO,ARAVENA AGUILA,1964-11-04 00:00:00,M,EDUARDO.E.ARAVENA@BHPBILLITON.COM
4,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,10035740-2,12079944,4,12079944-4,CÓNYUGE,ROSA,SALAS ALLENDE,1969-04-30 00:00:00,F,NaN
5,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,10053600-5,10053600,5,10053600-5,TITULAR,MAURICIO ALFONSO,GARCIA ROBLES,1965-05-05 00:00:00,M,MAURICIO.MG.GARCIA@BHPBILLITON.COM
6,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,10100025-7,10100025,7,10100025-7,TITULAR,LORENA PATRICIA,GONZALEZ KRSTULOVIC,1970-07-24 00:00:00,F,LORENA.LP.GONZALEZ@BHP.COM
7,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,10124407-5,10124407,5,10124407-5,TITULAR,MARCELA A.,FUENTEALBA POBLETE,1966-09-10 00:00:00,F,MARCELA.MF.FUENTEALBA@BHPBILLITON.COM
8,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,10192083-6,10192083,6,10192083-6,TITULAR,DAVID ANTONIO,OLGUIN PEÑA,1967-10-14 00:00:00,M,DAVID.OLGUIN@BHPBILLITON.COM
9,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,10192083-6,9271597,3,9271597-3,CÓNYUGE,SUSANA,ALVAREZ MONTENEGRO,1963-08-19 00:00:00,F,NaN


In [15]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("–", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19544\4228745446.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


(22509, 25)

In [16]:
Pacientes_Archivo.head()

,Llave,Cog Convenio,Nombre Convenio,Poliza,Rut Contratante,Dv Contratante,Nombre Contratante,Rut Filial,Dv Filial,Empresa,...,Rut Asegurado,Dv Asegurado,rut1,Parentesco,nombres,apellidos,nacimiento,sexo,email,rut
0,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,10025371,2,10025371-2,TITULAR,VIVIANA,CORREA ELIAS,1967-03-04 00:00:00,F,VIVIANA.E.CORREA@GMAIL.COM,100253712
1,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,10028831,1,10028831-1,TITULAR,DANIELLI JOEL,GODOY JOFRE,1966-10-29 00:00:00,M,DANIELLI.DD.GODOY@BHP.COM,100288311
2,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,10934210,6,10934210-6,CÓNYUGE,RODE ESTER,GONZALEZ CISTERNAS,1968-06-09 00:00:00,F,NaN,109342106
3,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,10035740,2,10035740-2,TITULAR,EDUARDO,ARAVENA AGUILA,1964-11-04 00:00:00,M,EDUARDO.E.ARAVENA@BHPBILLITON.COM,100357402
4,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,12079944,4,12079944-4,CÓNYUGE,ROSA,SALAS ALLENDE,1969-04-30 00:00:00,F,NaN,120799444


In [17]:
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rut2"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("-", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("–", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.upper()
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.strip()
Pacientes_Archivo.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19544\980738941.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")


(22509, 26)

In [18]:
Pacientes_Archivo.head()

,Llave,Cog Convenio,Nombre Convenio,Poliza,Rut Contratante,Dv Contratante,Nombre Contratante,Rut Filial,Dv Filial,Empresa,...,Dv Asegurado,rut1,Parentesco,nombres,apellidos,nacimiento,sexo,email,rut,rutT
0,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,2,10025371-2,TITULAR,VIVIANA,CORREA ELIAS,1967-03-04 00:00:00,F,VIVIANA.E.CORREA@GMAIL.COM,100253712,100253712
1,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,1,10028831-1,TITULAR,DANIELLI JOEL,GODOY JOFRE,1966-10-29 00:00:00,M,DANIELLI.DD.GODOY@BHP.COM,100288311,100288311
2,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,6,10934210-6,CÓNYUGE,RODE ESTER,GONZALEZ CISTERNAS,1968-06-09 00:00:00,F,NaN,109342106,100288311
3,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,2,10035740-2,TITULAR,EDUARDO,ARAVENA AGUILA,1964-11-04 00:00:00,M,EDUARDO.E.ARAVENA@BHPBILLITON.COM,100357402,100357402
4,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,4,12079944-4,CÓNYUGE,ROSA,SALAS ALLENDE,1969-04-30 00:00:00,F,NaN,120799444,100357402


In [19]:
def siPrimerCero(y):
    if y == "00":
        return "Nones"
    if y[0] == "0":
        y = y[1:]
    return y

In [20]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].apply(lambda y: siPrimerCero(y))
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].apply(lambda y: siPrimerCero(y))

In [21]:
Pacientes_Archivo.shape

(22509, 26)

In [22]:
Pacientes_Archivo.drop_duplicates(
    [
        "rut",
        "rutT",
        "Poliza",
    ],
    keep="last",
)
Pacientes_Archivo.shape

(22509, 26)

In [23]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
    # Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].apply(lambda y: siPrimerCero(y))
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("–", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(".", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19544\982349716.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19544\982349716.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_19544\982349716.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_S

In [24]:
# aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(22509, 36)

In [25]:
Pacientes_con_id.head()

,Llave,Cog Convenio,Nombre Convenio,Poliza,Rut Contratante,Dv Contratante,Nombre Contratante,Rut Filial,Dv Filial,Empresa,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,True,0018c00002axSPyAAM,100253712,Viviana Correa Eliash,RUT,None,None,None,Mujer,None
1,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,True,0018c00002b0eBVAAY,100288311,Danielli Joel Godoy Jofre,RUT,None,None,None,Hombre,None
2,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,True,0018c00002bHqLYAA0,109342106,Rode Gonzalez Cisternas,RUT,None,None,None,Mujer,None
3,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,True,0018c00002bHVfBAAW,100357402,Eduardo Aravena Aguila,RUT,None,None,None,Hombre,None
4,70016330K281610,1,CARE ASSISTANCE Plan BASICO,281610,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,70016330,K,CAJA DE COMPENSACION DE ASIGNACION FAMILIAR LO...,...,True,0018c00002bHsecAAC,120799444,Rosa Salas Allende,RUT,None,None,None,Mujer,None


In [26]:
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")


def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""

In [27]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )
""" if ("rutT" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Titulares"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    ) """

if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = Pacientes_con_id["email"].astype(str).str.lower()
    Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].apply(
        lambda y: isValid(y)
    )
    # Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].str.replace("no informado", "")
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "H": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "Prefiero no responder",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "M": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon": "Hombre",
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.lower().replace("nan", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("_", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("/", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("null", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NULL", "")


Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = True


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
14057,False,Wally Marleen,Lagos Sepulveda,136533991,,2019-08-31 00:00:00,Mujer,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [28]:
def traerId(rut):
    if len(rut) >= 8:
        busquedaDatos = "FIND {" + rut + "} RETURNING Account(Id,IdentificationId__pc)"
        derty = dict(sf.search(busquedaDatos))
        if derty["searchRecords"] != []:
            if derty["searchRecords"][0]["attributes"]["type"] == "Account":
                jote = derty["searchRecords"][0]["Id"]
                dertyd = derty["searchRecords"][0]["IdentificationId__pc"]
                print(jote + " " + dertyd)
                return jote
            else:
                print("no")
        else:
            print("vacio")
    else:
        return ""


def agregarPaciente(listado):
    vListado = listado.split(",")
    derty = dict(
        sf.Account.create(
            # derty = sf.Account.create(
            {
                "FirstName": vListado[0],
                "LastName": vListado[1],
                "IdentificationId__pc": vListado[2],
                "RecordTypeId": vListado[3],
                "BillingCountry": vListado[4],
                "Identification_Type__c": vListado[5],
                "Pa_s_del_Paciente__c": vListado[6],
                "Cuenta_validada__c": True,
                "PersonEmail": vListado[8],
                # "PersonBirthdate": datetime.strptime(vListado[9],'%Y-%m-%d %H:%M:%S' ),
                "HealthCloudGA__Gender__pc": vListado[9],
                "Phone": vListado[10],
                "BillingCity": vListado[11],
                "BillingState": vListado[12],
                "BillingStreet": vListado[13],
            }
        )
    )
    return derty["id"]

In [29]:
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
14057,False,Wally Marleen,Lagos Sepulveda,136533991,,2019-08-31 00:00:00,Mujer,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [30]:
Subir_pacientes["Id"] = Subir_pacientes["IdentificationId__pc"].apply(
    lambda y: traerId(y)
)
Subir_pacientes.drop_duplicates(
    [
        "IdentificationId__pc",
        
    ],
    keep="last",
)
Subir_pacientes.shape

vacio


(1, 13)

In [31]:
Subir_pacientes["dato_account"] = ""
if ("FirstName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + Subir_pacientes["FirstName"].astype(str)
if ("LastName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["LastName"].astype(str)
if ("IdentificationId__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["IdentificationId__pc"].astype(str)
if ("RecordTypeId" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["RecordTypeId"].astype(str)
if ("BillingCountry" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["BillingCountry"].astype(str)
if ("Identification_Type__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["Identification_Type__c"].astype(str)
if ("Pa_s_del_Paciente__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["Pa_s_del_Paciente__c"].astype(str)
if ("Cuenta_validada__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["Cuenta_validada__c"].astype(str)

if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["PersonEmail"].astype(str)
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," 
""" if ("PersonBirthdate" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["PersonBirthdate"].astype(str)
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","  """     
if ("HealthCloudGA__Gender__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["HealthCloudGA__Gender__pc"].astype(str)
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","  
if ("Phone" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["Phone"].astype(str)
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","   
if ("BillingCity" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["BillingCity"].astype(str)
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","  
if ("BillingState" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["BillingState"].astype(str)
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","  
if ("BillingStreet" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + "," + Subir_pacientes["BillingStreet"].astype(str) 
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","   

#Subir_pacientes["dato_account"] =  Subir_pacientes["dato_account"] =

Subir_pacientes["Id_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
)



In [32]:
query2 = "select P_liza__c, Nombre_de_Empresa__c, Rut_Empresa__c, Empresa_Padre__r.Name, Empresa__r.name,Holding__c, Plan__c, Pais__c, Activo__c, Id from Datos_Empresa__c "

df2 = sf.smart_query(query2)
df2.to_excel("F:/Empresas SF/reportes/empresas_nuevas.xlsx", index=False)
df2.head()

100%|██████████| 1/1 [00:10<00:00, 10.63s/it]


,P_liza__c,Nombre_de_Empresa__c,Rut_Empresa__c,Empresa_Padre__r.Name,Holding__c,Plan__c,Pais__c,Activo__c,Id
0,Sin Póliza Asociada,Colaboradores Care Assistance Chile SPA,775173998,Colaboradores,Siempre Chile SPA,Plan Gold,Chile,True,a3yRN0000006mmjYAA
1,-,Apunto Servicios De Alimentacion,-,Consalud,Holding Consalud,-,Chile,False,a3yRN0000006rG9YAI
2,-,2M Inmobiliaria Spa.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rGAYAY
3,-,360 Gms Capacitacion Spa.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rGBYAY
4,-,361 Gms Capacitacion Spa.,-,THB Seguros COVID,Solo Titulares,-,Chile,True,a3yRN0000006rGCYAY


In [33]:
Subir_pacientes.to_excel("F:/Empresas SF/reportes/111account.xlsx", index=False)

In [34]:
""" Subir_pacientes["Id"] = Subir_pacientes["IdentificationId__pc"].apply(
    lambda y: traerId(y)
) """
Subir_pacientes["dato_account"] = Subir_pacientes["FirstName"] + "," + Subir_pacientes["IdentificationId__pc"] + "," + Subir_pacientes["PersonEmail"] + "," 
Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + Subir_pacientes["PersonBirthdate"] + "," + Subir_pacientes["HealthCloudGA__Gender__pc"] + "," + Subir_pacientes["Cuenta_validada__c"] + "," 
Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + Subir_pacientes["Phone"] + "," + Subir_pacientes["BillingCity"] + "," + Subir_pacientes["BillingState"] + ","
Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + Subir_pacientes["BillingStreet"] + "," + Subir_pacientes["RecordTypeId"] + "," + Subir_pacientes["Identification_Type__c"] + "," +
Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + Subir_pacientes["BillingCountry"] + "," + Subir_pacientes["Pa_s_del_Paciente__c"] 


#sf.Contact.create({"LastName": "Smith", "Email": "example@example.com"})
Subir_pacientes.head()

SyntaxError: invalid syntax (895657022.py, line 7)

In [ ]:
derty = Subir_pacientes["IdentificationId__pc"]
sf.Contact.create({"IdentificationId__pc": "derty", "Email": "example@example.com"})

In [ ]:
Subir_pacientes.head()

In [ ]:
Subir_pacientes.head()
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account__l.xlsx", index=False)

In [ ]:
""" if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )
 """

In [ ]:
""" if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    ) """

In [ ]:
# apprix_1 = Pacientes_SF.iloc[:790552,:]
# apprix_2 = Pacientes_SF.iloc[790553:,:]
# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)

In [ ]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)

In [ ]:
campaña_en_pacientes2["rut"].shape

In [ ]:
# nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
# campaña = "BCI Seguros Colectivos"
# Pais = "Chile"
# Pais = "México"

# rutes = pd.read_excel(nuevosrutes)
rutes = pd.DataFrame(columns=["rut"])
rutes["rut"] = Pacientes_Archivo["rut"]
rutes["rutT"] = Pacientes_Archivo["rut2"]
rutes["Contratante"] = Pacientes_Archivo["Contratante"]
# rutes["poliza"] = Pacientes_Archivo["poliza"]
rutes.head()

In [ ]:
rutes["rut"] = rutes["rut"].astype(str)
rutes["rut"] = rutes["rut"].str.replace(".", "")
rutes["rut"] = rutes["rut"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["rut"] = rutes["rut"].str.upper()
rutes["rut"] = rutes["rut"].str.strip()

rutes["rutT"] = rutes["rutT"].astype(str)
rutes["rutT"] = rutes["rutT"].str.replace(".", "")
rutes["rutT"] = rutes["rutT"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["rutT"] = rutes["rutT"].str.upper()
rutes["rutT"] = rutes["rutT"].str.strip()

""" rutes["poliza"] = rutes["poliza"].astype(str)
rutes["poliza"] = rutes["poliza"].str.replace(".", "")
rutes["poliza"] = rutes["poliza"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["poliza"] = rutes["poliza"].str.strip() """


rutes.head()

In [ ]:
# derty = rutes.to_numpy().transpose().tolist()


Pacientes_SF2 = Pacientes_SF[["Rut", "Id Account Paciente"]]

In [ ]:
rutesi = pd.merge(
    left=rutes,
    right=Pacientes_SF2,
    how="left",
    left_on="rut",
    right_on="Rut",
)
rutesi.shape
rutesi.to_excel("F:/Empresas SF/reportes/rutsyempresa.xlsx", index=False)

In [ ]:
rutesi.head()

In [ ]:
rutesi_antg = pd.merge(
    left=rutesi,
    right=empresas_unidas,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)

In [ ]:
rutesi_antg.head()

In [ ]:
# rutesi_antg .to_excel("F:/Empresas SF/reportes/ruts_empresas_antiguos.xlsx", index=False)

In [ ]:
import json
import pandas as pd
from simple_salesforce import Salesforce, format_soql, format_external_id
from json import loads, dumps
import simpler_sf

simpler_sf.simple_salesforce()

In [ ]:
with open("login.json", "r") as j:
    misDatos = json.load(j)
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)

In [ ]:
""" query = "select Id, Account.Name, RutContact__c from AccountContactRelation "

df1 = sf.smart_query(query) """

In [ ]:
# nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
# campaña = "BCI Seguros Colectivos"
# Pais = "Chile"
# Pais = "México"

# rutes = pd.read_excel(nuevosrutes)
# rutes = Subir_pacientes["rut"]

In [ ]:
""" df1.head() """

In [ ]:
""" df1["rut"] = df1["RutContact__c"].astype(str)
df1["rut"] = df1["rut"].str.replace(".", "")
df1["rut"] = df1["rut"].str.replace(
    "-", ""
)  # aca se trabaja con el nuevo rut desde rut1 ojooooooo
df1["rut"] = df1["rut"].str.upper()
df1["rut"] = df1["rut"].str.strip()
df1.head() """

In [ ]:
""" rutes.head() """

In [ ]:
""" rutesi = pd.merge(
    left=rutes,
    right=df1,
    how="left",
    left_on="rut",
    right_on="rut",
) """

In [ ]:
""" rutesi.to_excel("F:/Empresas SF/reportes/rutesnuevos_accountcontactrelation.xlsx", index=False) """

In [ ]:
""" df2["P_liza__c"] = df2["P_liza__c"].astype(str)
df2["P_liza__c"] = df2["P_liza__c"].str.replace(".", "")
df2["P_liza__c"] = df2["P_liza__c"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
df2["P_liza__c"] = df2["P_liza__c"].str.upper()
df2["P_liza__c"] = df2["P_liza__c"].str.strip() """

In [ ]:
modelo_nuevo = pd.merge(
    left=rutesi,
    right=df2,
    how="left",
    left_on="poliza",
    right_on="P_liza__c",
)
modelo_nuevo.to_excel("F:/Empresas SF/reportes/modelo_nuevo.xlsx", index=False)

In [ ]:
modelo = pd.DataFrame()
modelo = modelo_nuevo[["rutT", "Id Account Paciente", "Id"]]

In [ ]:
modelo.head()